In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim 
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
from torch.utils.data import DataLoader,Dataset
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.decomposition import PCA

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [4]:
train

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,1000.0,12.5,90,85,280,90,16,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,1000.0,12.5,90,100,280,90,16,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [5]:
# 절반 이상이 NaN인 데이터 column 드롭
drop_col = train.loc[:,(train.isna().sum(axis=0)>=len(train)/2).values].columns
train.drop(columns = drop_col,inplace = True)
train

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,1000.0,12.5,90,85,280,90,16,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,1000.0,12.5,90,100,280,90,16,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [6]:
# 전부 다 다른 column과 전부 다 같은 column도 드롭
print(len(train.columns[(train.nunique()==1)|(train.nunique()==len(train))]))
train.drop(columns=train.columns[(train.nunique()==1)|(train.nunique()==len(train))],inplace=True)

35


In [7]:
fill_col = train.columns[train.isna().sum(axis=0)>0]
fill_col

Index(['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'],
      dtype='object')

In [8]:
no_agreement_col_list = train.columns[(train.applymap(lambda x: type(x).__name__).nunique()>1).values]
no_agreement_col_list

Index(['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'],
      dtype='object')

In [9]:
# 불일치 컬럼의 dtype 조사
# OK 부분이 잘못됐음
condition = (train.loc[:,no_agreement_col_list] != 'OK')

In [10]:
replace_list = []
replace_list.append(train.loc[condition.iloc[:,0],no_agreement_col_list[0]].dropna().astype(float).mean())
replace_list.append(train.loc[condition.iloc[:,1],no_agreement_col_list[1]].dropna().astype(float).mean())
replace_list.append(train.loc[condition.iloc[:,2],no_agreement_col_list[2]].dropna().astype(float).mean())

In [11]:
print(replace_list)

[465.18351067063895, 838.0938773028515, 720.05891651973]


In [12]:
train.loc[:,no_agreement_col_list[0]] = train.loc[:,no_agreement_col_list[0]].replace({'OK':replace_list[0]})
train.loc[:,no_agreement_col_list[1]] = train.loc[:,no_agreement_col_list[1]].replace({'OK':replace_list[1]})
train.loc[:,no_agreement_col_list[2]] = train.loc[:,no_agreement_col_list[2]].replace({'OK':replace_list[2]})

In [13]:
# 결측치 평균으로 제거
train.loc[:,no_agreement_col_list[0]].fillna(replace_list[0],inplace=True)
train.loc[:,no_agreement_col_list[1]].fillna(replace_list[1],inplace=True)
train.loc[:,no_agreement_col_list[2]].fillna(replace_list[2],inplace=True)

In [14]:
train[no_agreement_col_list[0]] = train.loc[:,no_agreement_col_list[0]].astype(float)
train[no_agreement_col_list[1]] = train.loc[:,no_agreement_col_list[1]].astype(float)
train[no_agreement_col_list[2]] = train.loc[:,no_agreement_col_list[2]].astype(float)

In [15]:
print(train.isna().sum().sum())

0


In [16]:
train.reset_index(inplace=True,drop=True)

In [17]:
train

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,1000.0,12.5,90,85,280,90,16,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,1000.0,12.5,90,100,280,90,16,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [18]:
train.to_csv('preprocessing_train.csv',index=False)

# Test Set 성능 측정

In [19]:
test = pd.read_csv('./data/test.csv')
test = test[train.columns]
test

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #2,AJX75334501,3J1XF767-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,19.8,13.0,195,1,0,NaN
1,Dam dispenser #2,AJX75334501,4B1XD472-2,1000.0,12.5,90,70,280,90,16,...,50.0,91.8,270,50,85.0,19.8,14,256,1,NaN
2,Dam dispenser #1,AJX75334501,3H1XE355-1,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.7,1.0,98,1,0,NaN
3,Dam dispenser #2,AJX75334501,3L1XA128-1,1000.0,12.5,90,70,280,90,10,...,50.0,91.8,270,50,85.0,20.0,14,0,1,NaN
4,Dam dispenser #1,AJX75334501,4A1XA639-1,240.0,2.5,-90,70,1030,-90,16,...,50.0,91.8,270,50,85.0,19.8,1,215,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,Dam dispenser #2,AJX75334501,3K1XB597-1,1000.0,12.5,90,70,280,90,10,...,50.0,91.8,270,50,85.0,19.5,14,131,1,NaN
17357,Dam dispenser #2,AJX75334501,4A1XB974-1,1000.0,12.5,90,70,280,90,16,...,50.0,91.8,270,50,85.0,19.8,12,279,1,NaN
17358,Dam dispenser #1,AJX75334501,3L1XA998-1,240.0,2.5,-90,70,1030,-90,16,...,50.0,91.8,270,50,85.0,20.5,4,66,1,NaN
17359,Dam dispenser #1,AJX75334501,3F1XC376-1,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,18.9,1.0,117,1,0,NaN


In [20]:
train.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [21]:
no_agreement_col_list = test.columns[(test.applymap(lambda x: type(x).__name__).nunique()>1).values]
no_agreement_col_list

Index(['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'],
      dtype='object')

In [22]:
condition = ~(test.loc[:,no_agreement_col_list].isin(['OK','NG']))
condition

,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2
0,True,True,True
1,False,False,False
2,True,True,True
3,True,True,True
4,False,False,False
...,...,...,...
17356,True,True,True
17357,False,False,False
17358,True,True,True
17359,True,True,True


In [23]:
replace_list = []
replace_list.append(test.loc[condition.iloc[:,0],no_agreement_col_list[0]].dropna().astype(float).mean())
replace_list.append(test.loc[condition.iloc[:,1],no_agreement_col_list[1]].dropna().astype(float).mean())
replace_list.append(test.loc[condition.iloc[:,2],no_agreement_col_list[2]].dropna().astype(float).mean())

In [24]:
test.loc[:,no_agreement_col_list[0]] = test.loc[:,no_agreement_col_list[0]].replace({'OK':replace_list[0]})
test.loc[:,no_agreement_col_list[1]] = test.loc[:,no_agreement_col_list[1]].replace({'OK':replace_list[1]})
test.loc[:,no_agreement_col_list[2]] = test.loc[:,no_agreement_col_list[2]].replace({'OK':replace_list[2]})

In [25]:
# 결측치 평균으로 제거
test.loc[:,no_agreement_col_list[0]].fillna(replace_list[0],inplace=True)
test.loc[:,no_agreement_col_list[1]].fillna(replace_list[1],inplace=True)
test.loc[:,no_agreement_col_list[2]].fillna(replace_list[2],inplace=True)

In [26]:
test[no_agreement_col_list[0]] = test.loc[:,no_agreement_col_list[0]].astype(float)
test[no_agreement_col_list[1]] = test.loc[:,no_agreement_col_list[1]].astype(float)
test[no_agreement_col_list[2]] = test.loc[:,no_agreement_col_list[2]].astype(float)

In [27]:
test.to_csv('./preprocessing_test.csv',index=False)